### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [1]:
PAGE_FORMATS = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}


class ReadableEntity:
    def __init__(self, author, num_pages, page_format, text, rarity):
        self.author = author
        self.num_pages = num_pages
        self.page_format = page_format
        self.text = text
        self.rarity = rarity
    
    @staticmethod
    def get_page_square(heigth, width):
        return heigth * width


class Journal(ReadableEntity):
    def __init__(self, volume, *kwargs):
        self.volume = volume
        super().__init__(*kwargs)
    
    @property
    def get_journal_square(self): # договорились на семинаре, что книги не рвём
        heigth, width = PAGE_FORMATS[self.page_format.upper()]
        return self.num_pages * self.get_page_square(heigth, width)

    
class Book(ReadableEntity):
    def __init__(self, hard_cover, *kwargs):
        self.hard_cover = hard_cover
        super().__init__(*kwargs)

        
class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key]))
        print('Entity was successfully saved')
     
    
class LibraryJournal(Journal, Exporter):
    def __init__(self, *kwargs):
        super().__init__(*kwargs)    
        
        if self.rarity >= 9:
            self.export_to_txt()


class LibraryBook(Book, Exporter):
    def __init__(self, *kwargs):
        super().__init__(*kwargs)    
        
    def process_book(self):
        if self.rarity >= 9:
            self.export_to_txt(self.author+'.txt')
        else:
            print('Entity is not rare')

In [2]:
tatler = Journal("02/2019", "Conde Nast", 220, "A4", "style", 2)
tatler.get_journal_square

In [3]:
library_freud = LibraryBook(True, "Freud", 144, "A4", "lorem ipsum", 9)
library_freud.process_book()

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [4]:
import sys
import collections

def total_size(obj):
    
    if isinstance(obj, (int, float, complex, bool, str)):
        return sys.getsizeof(obj)
    
    if isinstance(obj, dict):
        cnt = 0
        for key, value in obj.items():
            cnt += total_size(key)
            cnt += total_size(value)
        return cnt
    
    if isinstance(obj, collections.Iterable):
        cnt = 0
        for elem in obj:
            cnt += total_size(elem)
        return cnt

In [5]:
total_size(1)

28

In [6]:
total_size("abc")

52

In [7]:
total_size(1.2)

24

In [8]:
total_size(['a','b'])

100

In [9]:
total_size(('a', 'b'))

100

In [10]:
total_size(dict({'a': ['b', 'b']}))

150

### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [11]:
import math

class Monom(object): 
    
    def __init__(self, coef, power):
        '''
            Класс для работы с одним мономом многочлена
        '''
        self.coef = coef
        self.power = power
        
    def __call__(self, value):
        return self.coef * (value ** self.power)
    
    def __eq__(self, other):
        return (self.coef == other.coef) & (self.power == other.power)
        
        
    def __add__(self, other): 
        if self.power == other.power:
            return Monom(self.coef + other.coef, self.power)
        
    def __sub__(self, other):
        if self.power == other.power:
            return Monom(self.coef - other.coef, self.power)
    
    
    def __mult__(self, other):
        return Monom(self.coef * other.coef, self.power + other.power)
    
    def __truediv__(self, other):
        return Monom(self.coef / other.coef, self.power - other.power)
    
    
    def __repr__(self): 
        coef_str = ""
        power_str = ""
        
        if (self.coef == 0):
            return "0"
        elif (self.coef == 1) & (self.power != 0):
            coef_str = ""
        elif (int(self.coef.real) != self.coef.real) | isinstance(self.coef, complex):
            coef_str = str(self.coef)
        else:
            coef_str = str(int(self.coef))
            
        if self.power == 0:
            power_str = ""
        elif self.power == 1:
            power_str = "x"
        elif ((self.power < 0) | (int(self.power) != self.power)):
            power_str = "x^({})".format(self.power)
        else:
            power_str = "x^{}".format(int(self.power))
        
        return coef_str + power_str ###### константа
    
    def __derivative__(self):
        return Monom(self.coef * self.power, self.power - 1)
    
    def __integral__(self):
        return Monom(float(self.coef) / (self.power + 1), self.power + 1)

In [12]:
class Polynomial (object):
    
    @staticmethod
    def parse_coeffs(coefs):
        monoms = []
        
        if isinstance(coefs[0], tuple):
            obj_to_iterate = sorted(coefs, key=lambda t: -t[1])
        else:
            obj_to_iterate = zip(coefs, reversed(range(len(coefs))))
        
        for coef, power in obj_to_iterate:
            if (coef == 0) & ((power != 0) | (len(monoms) > 0)):
                continue
            monoms.append(Monom(coef, power))
        return monoms

    def __init__(self, coefs):
        '''
            Создаем из набора упорядоченных коэффициентов полином.
            Polynomial ([]) -> P = 0
            Polynomial ([2, -4.2, 1]) -> P = 2x^2-4.2x+1
        '''
        # Create a polynomial from a list or array of coefficients
        
        self._raw_coefs = coefs
        
        self.monoms = self.parse_coeffs(coefs)
        
        self.degree = self.monoms[0].power
        
        self.power_to_coef = {m.power: m.coef for m in self.monoms}
            
    def __call__(self, value): # 9 + 
        return sum([m.__call__(value) for m in self.monoms])
        
    def __repr__(self): # 6 + 
        string = self.monoms[0].__str__()
        
        for m in self.monoms[1:]:
            if m.coef >= 0:
                sign = "+"
            else:
                sign = ""
            string += sign + m.__str__()
            
        return string
    
    def __neg__(self):
        return Polynomial([(-m.coef, m.power) for m in self.monoms])
    
    def __add__(self, other): # 1 + 
        statedict = self.power_to_coef.copy()
        
        for o in other.monoms:
            coef, power = o.coef, o.power
                    
            if power not in statedict:
                statedict[power] = coef
            else:
                statedict[power] += coef
                    
        return Polynomial([(coef, power) for power, coef in statedict.items()])
        
    def __sub__(self, other): # 2 + 
        return self + other.__neg__()
    
    @staticmethod
    def z_t(t, N):
        return math.cos(2*math.pi*t / N) + 1j * math.sin(2*math.pi*t / N)
    
    def __mul__(self, other): # 3a +
         # https://neerc.ifmo.ru/trains/toulouse/2017/fft.pdf
        N = self.degree * other.degree
        
        z = [self.z_t(i, N) for i in range(N)]

        h = [self.__call__(z_t) * other.__call__(z_t) for z_t in z]
        p_h = Polynomial(h[1:][::1] + [h[0]])

        return Polynomial([round(p_h(z_t).real / N, 7) for z_t in (reversed(z))])
        
    def __truediv__(self, other): # 4
        pass
    
    def __pow__(self, power): # 5
        pass # умножение за логарифм
    
    def __pow__(self, n):
        N = self.degree ** n
        
        z = [self.z_t(i, N) for i in range(N)]

        h = [self.__call__(z_t) ** n for z_t in z] ## спасибо Фурье за это
        p_h = Polynomial(h[1:][::1] + [h[0]])

        return Polynomial([round(p_h(z_t).real / N, 6) for z_t in (reversed(z))])
    
    def derivative(self): # 7 + 
        parts = [m.__derivative__() for m in self.monoms]
        return Polynomial([(d.coef, d.power) for d in parts])
    
    def integral(self, *args): # 8 +
        parts = [m.__integral__() for m in self.monoms]
        poly = Polynomial([(d.coef, d.power) for d in parts]) 
        
        if len(args) == 0:
            return poly
        elif len(args) == 2:
            a, b = args
            return poly(b) - poly(a)
        else:
            raise AttributeError

In [13]:
p1 = Polynomial([2, 0, 4, 1])
p1

2x^3+4x+1

In [14]:
p2 = Polynomial([2, 4, 1])
p2

2x^2+4x+1

In [15]:
p1 + p2 # 1. сложение

2x^3+2x^2+8x+2

In [16]:
p1 - p2 # 2. вычитание

2x^3-2x^2

In [17]:
p1 * p2 # 3а. перемножение с Фурье

4x^5+8x^4+10x^3+18x^2+8x+1

In [18]:
p1 ** 3  # 5. возведение в степень

8x^9+48x^7+12x^6+96x^5+48x^4+70x^3+48x^2+12x+1

In [19]:
p1 # 6. __repr__ 

2x^3+4x+1

In [20]:
p1.derivative() # 7. производная

6x^2+4

In [21]:
p1.integral() # 8. интеграл

0.5x^4+2x^2+x

In [22]:
p1.integral(0, 1) # 8* - определенный интеграл

3.5

In [23]:
p1(0) # 9. значение в точке

1

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [25]:
class Iterator():
    def __init__(self, *args):
        
        left = 0
        step = 1
        
        for elem in args:
            if not isinstance(elem, int):
                raise TypeError("'float' object cannot be interpreted as an integer")
        
        if len(args) == 1:
            right = args[0]
        elif len(args) == 2:
            left, right = args
        elif len(args) == 3:
            left, right, step = args
        else:
            raise TypeError('range expected at most 3 arguments, got {}'.format(len(args)))
        
        self.left, self.right, self.step = left, right, step
        
        self.count = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        current = self.left + self.count
        self.count += self.step
        
        if self.step * (current - self.right) >= 0:
            raise StopIteration
            
        return current

In [26]:
list(Iterator(1)) == list(range(1)) 

True

In [27]:
list(Iterator(1, 3)) == list(range(1, 3)) 

True

In [28]:
list(Iterator(1, 5, 2)) == list(range(1, 5, 2)) 

True

In [29]:
list(Iterator(1, 5, 2)) == list(range(1, 5, 2)) 

True

In [30]:
list(Iterator(5, 1, -1)) == list(range(5, 1, -1)) 

True

In [31]:
range(3, 3, 4, 5)

TypeError: range expected at most 3 arguments, got 4

In [32]:
Iterator(3, 3, 4, 5)

TypeError: range expected at most 3 arguments, got 4

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [33]:
class PositiveSmallIntegerField:
    
    def __init__(self, name):
        self.name = name

    def __get__(self, obj, real):
        return obj.__dict__[self.name]
    
    def __set__(self, obj, real):
        
        if (real < 0) | (real > 32767):
            raise ValueError('Integer must be between 0 and 32767')

        obj.__dict__[self.name] = real

class SmallInteger:
    real = PositiveSmallIntegerField('real')
    
    def __init__(self, real):
        self.real = real
        
    def __repr__(self):
        return str(self.real)
    
    def __add__(self, other):
        return SmallInteger(self.real+other.real)

In [34]:
a = SmallInteger(10000)

In [35]:
a + 1

10001

In [36]:
a + 40000

ValueError: Integer must be between 0 and 32767

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [37]:
import time

class Catchtime:
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, type, value, traceback):
        print('{} seconds'.format(round(time.time() - self.start_time, 7)))

with Catchtime() as t2:
    time.sleep(4.)

4.0031319 seconds
